In [37]:
import torch
import sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import os

#INPUT_PATH = "data/train_conll_spanglish.csv"
INPUT_PATH = "data/train_conll_hinglish.csv"
MAX_TWEET = 280

char_to_ind = {}
ind_to_char = {}

char_to_ind.update({"UNK":0})
ind_to_char.update({0:"UNK"})

count = 1

with open(INPUT_PATH, 'r') as f:
    for line in f:
        for char in line.split('\t')[1]:
            if char.lower() not in char_to_ind:
                char_to_ind.update({char.lower():count})
                ind_to_char.update({count:char.lower()})
                count += 1

#print(char_to_ind)
#print(ind_to_char)

n_letters = len(char_to_ind)

In [38]:
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][char_to_ind[letter]] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(len(line), n_letters)
    for li, letter in enumerate(line):
        tensor[li][char_to_ind[letter]] = 1
    return tensor

#def tensorToLine(tensor):
#    ind_to_char[ind=position of the 1 in the tensor] = character
#    append character to the input line character

def batchToTensor(batch):
    tensor = torch.zeros(len(batch),MAX_TWEET,n_letters)
    for sentence, line in enumerate(batch):
        for li, letter in enumerate(line):
            tensor[sentence][li][char_to_ind[letter.lower()]] = 1
    return tensor


#print(letterToTensor('o'))
print(lineToTensor('hello how are tou').shape)
#print(batchToTensor(['hello friend', 'linear svm is better']))
#print(batchToTensor(['hello friend', 'linear svm is better']).shape)

torch.Size([17, 898])


In [39]:
trainpath = os.path.join("data", "train_conll_hinglish.csv")
train = pd.read_csv(trainpath, sep='\\t', names=["ID","SENTENCE","LABEL"])
test = pd.read_csv("sample_test.csv", names=["ID", "SENTENCE", "LABEL"])

/Users/nehajoshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [40]:
print(train['SENTENCE'][0].lower())
train_char_features = torch.sum(batchToTensor(train['SENTENCE']),1)

@ adilnisarbutt pakistan ka ghra tauq he pakistan israel ko tasleem nahein kerta isko palestine kehta he- occupied palestine


In [41]:
char_features = train_char_features
char_features = pd.DataFrame(char_features)
labels = pd.Series.as_matrix(train.LABEL)
#torch.tensor(m)

/Users/nehajoshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
model_NB = MultinomialNB()

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf, sent_train, sent_test = train_test_split(char_features, labels, train['SENTENCE'], train_size=0.8)
# X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(char_features, train.LABEL, train_size=0.75)
model_NB.fit(X_train_tfidf, y_train_tfidf)
predictions_tfidf = model_NB.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, predictions_tfidf)

In [43]:
precision, recall, f1, _ = precision_recall_fscore_support(y_test_tfidf, predictions_tfidf, average='macro')
print("[NB] accuracy: {}, f1-score: {}, precision: {}, recall: {}".format(accuracy_tfidf, f1, precision, recall))
print(accuracy_tfidf)
print(list(sent_test.keys())[10])
print(sent_test[list(sent_test.keys())[10]])
print(predictions_tfidf[1])
print(y_test_tfidf[1])

[NB] accuracy: 0.4757185332011893, f1-score: 0.472872604189309, precision: 0.4826296920488485, recall: 0.4912570786195222
0.4757185332011893
2800
Sir ghoomta hai maza ata hai rat me sapny ni aty neend achi ati hai ik dum mast- ft Kamlesh https// t co/ UhSt8pqEAo
positive
positive


In [9]:
def trainSVM(X_train_tfidf, y_train_tfidf, X_test_tfidf, y_test_tfidf):
    clf = SVC(kernel='linear', gamma='auto')
    clf.fit(X_train_tfidf, y_train_tfidf)
    predictions_svc = clf.predict(X_test_tfidf)
    accuracy_svc = accuracy_score(y_test_tfidf, predictions_svc)
    # accuracy_svc
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_tfidf, predictions_svc, average='macro')
    print("[Linear SVM] accuracy: {}, f1-score: {}, precision: {}, recall: {}".format(accuracy_svc, f1, precision, recall))

In [10]:
# 'linear' shows higher accuracy than 'rbf' (default)
clf = SVC(kernel='linear', gamma='auto')
clf.fit(X_train_tfidf, y_train_tfidf)
predictions_svc = clf.predict(X_test_tfidf)
accuracy_svc = accuracy_score(y_test_tfidf, predictions_svc)
# accuracy_svc
precision, recall, f1, _ = precision_recall_fscore_support(y_test_tfidf, predictions_svc, average='macro')
print("[Linear SVM] accuracy: {}, f1-score: {}, precision: {}, recall: {}".format(accuracy_svc, f1, precision, recall))

[Linear SVM] accuracy: 0.49008723235527357, f1-score: 0.4926617278274259, precision: 0.49585063252840644, recall: 0.4928409160780906


In [25]:
print(predictions_svc[10])
print(y_test_tfidf[10])

negative
negative


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
X = train_char_features
y = train.LABEL
skf = StratifiedKFold()
skf.get_n_splits(X, y)
print(skf)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("split")
    trainSVM(X_train, y_train, X_test, y_test)